# Extraction du nom et de la ref des auteurs

## Remarques 
* Extraction des noms non directement liées avec la ref_auteur dans le df, ref_noms != df['nom_auteur']
* Exception et bizarreries possible liées à l'extratction à la pisse(6 pourcent)

In [2]:
#import
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
data_path = "../data/"

In [3]:
# Dataframe des questions avec leurs principaux attributs
questions_df_2017 = pd.read_pickle("{}normalized_df_2017to2019_j.pkl".format(data_path))
questions_df_2017.head()

,date_question,date_reponse,groupe_auteur,ministere_adresse,question,rubrique,titre,uid
0,2020-04-14,NaT,NI,Ministère de la justice,m. ludovic pajot attire attention mme garde sc...,terrorisme,libération anticiper détenu terroriste radicalisé,QANR5L15QE28467
1,2019-04-23,NaT,LAREM,Ministère de la justice,m. jean-charles colas - roy attire attention m...,droits fondamentaux,reconnaissance langue signe français constitution,QANR5L15QE19021
2,2018-06-12,2018-12-25,LAREM,Ministère de l’intérieur,m. bertrand sorre attire attention m. ministre...,sécurité routière,précision 80 kilomètre heure,QANR5L15QE9330
3,2020-04-21,NaT,LAREM,Ministère des solidarités et de la santé,m. christophe blanchet interroger m. ministre ...,assurance complémentaire,frais mutuelle santé,QANR5L15QE28523
4,2018-06-19,2018-10-02,LAREM,Ministère de la transition écologique et solid...,mme françoise dumas attire attention m. minist...,eau et assainissement,aide agence eau projet réhabilitation disposit...,QANR5L15QE9466


## Extraction des auteurs

In [ ]:
# Extraction des données 

import json
import urllib.request
from zipfile import ZipFile
import os
import datetime as dt
import pandas as pd
import numpy as np
from io import StringIO, BytesIO
import shutil

def extraire_df(dossier_questions):
    data = []
    files = os.listdir(dossier_questions)
    nb = len(files)
    print("{} lettres à extraire ...".format(nb))
    for i, file in enumerate(files):
        if i%(nb//10) == 0 :
            print("{}% ... ".format(round(100*i/nb)))

        try:
            with open(dossier_questions+'/'+file, 'r') as f:
                # Code de Xavier Tannier pour ouvrir les fichiers avec le bon encodage
                content = f.read()
                io = StringIO(content)
                json_data = json.load(io)

                # print(json.dumps(json_data, indent=4, sort_keys=True))
                ref_auteur = json_data["question"]["auteur"]["identite"]["acteurRef"]
                uid = json_data["question"]["uid"]
                titre = json_data["question"]["indexationAN"]["ANALYSE"]["ANA"]
                # On ajoute les colonnes qui nous intéressent dans une liste de dictionnaire : data
                data.append({
                    "uid" : uid,
                    "ref_auteur": ref_auteur,
                    "titre":titre,
                })
        except Exception as e:
            print(e)
            pass

    return pd.DataFrame(data)
auteurs = extraire_df(data_path+"json")

In [72]:
# On associe chaque lettre du df normalisé à la référence de son auteur
auteurs_in = auteurs.set_index('uid').loc[questions_df_2017.set_index('uid').index] # Auteurs utilisés dans le questions df
dfna = questions_df_2017.set_index('uid') # On met uid en index
dfna.loc[auteurs_in.index,'ref_auteur'] = auteurs_in.ref_auteur.values # ON asssocie les références des auteurs
df =  dfn.sort_values(by='date_question').reset_index() # ON prend le nouveau df avec les ref_auteurs
df.head()

,uid,date_question,date_reponse,groupe_auteur,ministere_adresse,question,rubrique,titre,ref_auteur
0,QANR5L15QE16,2017-07-04,2017-12-26,LC,Ministère de la culture,m. meyer habib attire attention mme ministre c...,Français de l'étranger,impossibiliter acheter livre numérique françai...,PA695100
1,QANR5L15QE1,2017-07-04,2017-12-05,LC,Ministère de l’intérieur,m. antoine herth attire attention m. ministre ...,administration,difficulté administratif lier internet,PA267355
2,QANR5L15QE23,2017-07-04,2018-04-17,LC,Ministère des solidarités et de la santé,m. stéphane demilly attire attention mme minis...,maladies,maladie cœliaqu,PA267241
3,QANR5L15QE31,2017-07-04,2017-12-26,LR,Ministère des solidarités et de la santé,m. martial saddier attire attention mme minist...,sécurité sociale,double affiliation travailleur frontalier,PA267527
4,QANR5L15QE18,2017-07-04,2017-10-17,LR,"Secrétariat d'État auprès du Premier ministre,...",m. martial saddier attire attention mme secrét...,handicapés,difficulté accès déficient visuel certains erp,PA267527


In [ ]:
# Association des auteurs à un nom
df['nom_auteur'] = df.apply(lambda x : ' '.join(x['question'].split(' ')[1:3]),axis= 1)
ref_noms = df[['nom_auteur','ref_auteur','groupe_auteur']].groupby('ref_auteur').first()

In [222]:
ref_noms.head()

,nom_auteur,groupe_auteur
ref_auteur,,
PA1008,alain david,NG
PA1012,charles verpillière,LR
PA1029,bernard deflesselles,LR
PA1198,laurence dumont,NG
PA1206,nicolas dupont,NI
